In [14]:
# 作业三：从上面链接给定的语料库中均匀抽取200个段落（每个段落大于500个词），
# 每个段落的标签就是对应段落所属的小说。利用LDA模型对于文本建模，并把每个段落表示为主题分布后进行分类。
# 验证与分析分类结果，
# （1）在不同数量的主题个数下分类性能的变化；
# （2）以"词"和以"字"为基本单元下分类结果有什么差异？
import os
import glob
import random
import jieba
from sklearn.cluster import KMeans
num_paragraphs = 200  # 抽取段落数
min_length = 500  # 每个段落最小长度
basepath = 'C:\\Users\\HUAWEI\\Desktop\\深度学习与自然语言处理\\第三次作业'  #文件夹位置
with open(basepath+'\\jyxstxtqj_downcc.com\\inf.txt','r',encoding='ANSI') as f:
    booklist = f.read().split(',') #读目录 
with open(basepath+"\\cn_stopwords.txt",'r',encoding = 'utf-8-sig') as f:
    stopwords = f.read().split() #读停用词
stopwords2 = [' ','wwwcrcom','www','cr173','com','□','说道','说','道','便','笑','=','txt','\n','本书来自www.cr173.com免费txt小说下载站','更多更新免费电子书请关注www.cr173.com','\u3000','目录']
stopwords.extend(stopwords2)
content = []
for book in booklist:
    path = basepath+'\\jyxstxtqj_downcc.com\\'+ book +'.txt'
    with open(path, 'r', encoding='ANSI') as f:
        data_txt = f.read()  # 读取文本文件
    words = jieba.lcut(data_txt)  # 结巴分词
    words = [i for i in words if i not in stopwords]
    pos = int(len(words)//13)
    for i in range(13):
        data_temp = []
        data_temp = data_temp + words[i*pos:i*pos+500]
        content.append(data_temp)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\HUAWEI\AppData\Local\Temp\jieba.cache
DEBUG:jieba:Loading model from cache C:\Users\HUAWEI\AppData\Local\Temp\jieba.cache
Loading model cost 0.668 seconds.
DEBUG:jieba:Loading model cost 0.668 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [12]:
import numpy as np
import lda

# 208个段落，每个段落用一个list表示
paragraphs = content

# 构建词袋模型
vocab = set(word for paragraph in paragraphs for word in paragraph)
word2id = dict((word, id) for id, word in enumerate(vocab))
id2word = dict((id, word) for id, word in enumerate(vocab))
doc_word_matrix = np.zeros((len(paragraphs), len(vocab)), dtype=np.int32)
for i, paragraph in enumerate(paragraphs):
    for word in paragraph:
        doc_word_matrix[i, word2id[word]] += 1

# 训练LDA模型
model = lda.LDA(n_topics=10, n_iter=1000, random_state=1)
model.fit(doc_word_matrix)

# 输出每个主题的前10个关键词
topic_word = model.topic_word_
doc_topic = model.transform(doc_word_matrix)
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(list(vocab))[np.argsort(topic_dist)][:-(10+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

INFO:lda:n_documents: 208
INFO:lda:vocab_size: 27954
INFO:lda:n_words: 103931
INFO:lda:n_topics: 10
INFO:lda:n_iter: 1000
INFO:lda:<0> log likelihood: -1365081
INFO:lda:<10> log likelihood: -1118976
INFO:lda:<20> log likelihood: -1099758
INFO:lda:<30> log likelihood: -1091235
INFO:lda:<40> log likelihood: -1085040
INFO:lda:<50> log likelihood: -1079570
INFO:lda:<60> log likelihood: -1076526
INFO:lda:<70> log likelihood: -1073933
INFO:lda:<80> log likelihood: -1071630
INFO:lda:<90> log likelihood: -1069495
INFO:lda:<100> log likelihood: -1067399
INFO:lda:<110> log likelihood: -1065363
INFO:lda:<120> log likelihood: -1064224
INFO:lda:<130> log likelihood: -1063502
INFO:lda:<140> log likelihood: -1062132
INFO:lda:<150> log likelihood: -1060249
INFO:lda:<160> log likelihood: -1058904
INFO:lda:<170> log likelihood: -1058135
INFO:lda:<180> log likelihood: -1057054
INFO:lda:<190> log likelihood: -1056945
INFO:lda:<200> log likelihood: -1054764
INFO:lda:<210> log likelihood: -1054914
INFO:lda:

Topic 0: 韦小宝 陈家洛 萧中慧 康熙 卓天雄 周威信 张召重 皇帝 瞎子 公主
Topic 1: 范蠡 剑士 剑 长剑 剑法 青衣 勾践 阿青 少女 风
Topic 2: 一个 汉子 少年 中 老者 请 兵 这位 铁木真 罢
Topic 3: 令狐冲 黄蓉 派 中 穆念慈 欧阳克 仪琳 弟子 丐帮 武功
Topic 4: 中 皇帝 曰 中国 牋 百姓 杀 写 	 小说
Topic 5: 袁承志 胡斐 走 大汉 原来 青青 水笙 程灵素 见 知
Topic 6: 事 没 师父 想 知道 心中 见 做 瞧 会
Topic 7: 杀 听 中 想 姑娘 见到 一个 吃 张无忌 爷爷
Topic 8: 中 一声 听 见 只见 两人 一个 突然 出 二人
Topic 9: 麽 著 李文秀 後 於 曹云奇 苏普 甚 苏鲁克 爹爹


In [28]:
data = doc_topic
# 聚类为16类
kmeans = KMeans(n_clusters=16)
kmeans.fit(data)
# 输出每个样本所属的类别
labels = kmeans.labels_
for i in range(len(labels)):
    print(labels[i],end = ' ')
    if (i+1)%13 == 0:
        print('\n')


11 8 8 8 8 8 8 8 8 8 8 8 8 

9 0 0 0 0 3 0 0 8 3 3 9 9 

3 0 8 0 11 0 0 0 0 0 6 0 15 

11 13 13 13 11 2 0 0 3 3 4 13 13 

1 15 15 15 15 2 15 2 15 1 2 2 6 

9 9 6 1 9 1 9 1 1 9 9 1 9 

6 6 6 7 7 7 14 7 7 14 5 14 6 

1 7 3 0 4 6 7 11 6 13 7 7 3 

15 15 15 2 15 2 2 15 15 2 2 2 2 

6 12 7 12 11 7 12 6 14 7 14 1 5 

6 4 3 12 4 12 4 14 14 4 12 11 14 

6 14 7 7 4 7 7 14 14 14 7 4 5 

6 11 11 8 8 11 8 8 8 8 8 11 11 

1 6 13 13 12 12 12 5 6 12 7 6 7 

15 2 2 2 15 2 2 2 2 2 15 2 15 

10 10 10 10 10 10 10 10 10 4 10 10 4 



In [13]:
doc_topic

array([[2.56614444e-04, 3.89425876e-05, 1.73770979e-01, ...,
        2.99069098e-02, 3.93327489e-01, 2.39205257e-01],
       [2.70477760e-05, 1.33645354e-02, 6.86221977e-02, ...,
        1.48780946e-01, 2.47653716e-01, 4.27803685e-01],
       [5.79204507e-05, 4.74453598e-04, 2.09535331e-05, ...,
        1.54387987e-01, 2.51060327e-01, 4.33430149e-01],
       ...,
       [2.82304827e-05, 5.21755318e-01, 1.63566367e-05, ...,
        2.05442863e-01, 1.86600587e-01, 3.67164855e-05],
       [1.73661723e-05, 5.14646199e-01, 4.05896000e-05, ...,
        2.29692832e-01, 1.42804422e-01, 7.54460207e-05],
       [3.19739492e-05, 4.07046855e-01, 2.36167173e-02, ...,
        2.46954337e-01, 1.93264347e-01, 7.84564963e-05]])

In [20]:
import numpy as np
import random

doc_list=content
K=5
alpha=0.01
beta=0.1
num_iters=25
# 将列表中的元素转换为集合
word_dict = set(word for doc in doc_list for word in doc)
# 将列表中的元素转换为字典
word_dict = {word: i for i, word in enumerate(word_dict)}
# 将列表中的元素转换为numpy数组
doc_list = np.array([[word_dict[word] for word in doc] for doc in doc_list])
M = len(doc_list)  # 文档数
V = len(word_dict) # 词汇的大小
z = np.random.randint(0, K, size=(M, len(doc_list[0])))  # 主题分布
n_mz = np.zeros((M, K), dtype=int)  # 文档-主题分布
n_zv = np.zeros((K, V), dtype=int)  # 主题-词分布
n_z = np.zeros(K, dtype=int)  # 主题数量
for i in range(M):
    for j in range(len(doc_list[i])):
        k = z[i][j]
        n_mz[i][k] += 1
        n_zv[k][doc_list[i][j]] += 1
        n_z[k] += 1
# Gibbs采样
for iter in range(num_iters):
    for i in range(M):
        for j in range(len(doc_list[i])):
            k = z[i][j]
            n_mz[i][k] -= 1
            n_zv[k][doc_list[i][j]] -= 1
            n_z[k] -= 1
            p = (n_mz[i] + alpha) * (n_zv[:, doc_list[i][j]] + beta) / (n_z + V * beta)
            # 将代码向量化，以提高计算速度
            k = np.random.choice(K, p=p / p.sum())
            z[i][j] = k
            n_mz[i][k] += 1
            n_zv[k][doc_list[i][j]] += 1
            n_z[k] += 1

        
# 输出主题-词分布
for k in range(K):
    print('Topic {}:'.format(k+1))
    top5_positions = np.argsort(n_zv[k])[::-1][:5]
    for v in top5_positions:
        print('{}: {:.4f}'.format(list(word_dict.keys())[list(word_dict.values()).index(v)], n_zv[k][v] / n_z[k]))
print(n_mz)

Topic 1:
听: 0.0104
中: 0.0064
一个: 0.0059
想: 0.0052
爹爹: 0.0049
Topic 2:
韦小宝: 0.0086
杨过: 0.0043
弟子: 0.0041
张无忌: 0.0040
杀: 0.0038
Topic 3:
麽: 0.0094
中: 0.0072
见: 0.0064
一个: 0.0056
走: 0.0053
Topic 4:
见: 0.0066
陈家洛: 0.0058
武功: 0.0054
听: 0.0032
中: 0.0030
Topic 5:
范蠡: 0.0085
中: 0.0073
剑士: 0.0071
袁承志: 0.0064
长剑: 0.0042
[[120   1 323  56   0]
 [166   0 325   0   9]
 [126   5 353  16   0]
 ...
 [ 60   0  81   0 359]
 [ 86   0  76   0 338]
 [ 44   0 131   0 256]]


In [24]:
for i in range(len(n_mz)):
    print(n_mz[i])

[  0 182  73  60   2   0   1  34   0 148]
[  0 187   0  22   0  14  10 198   0  69]
[  0 196   0   6   0   0   0 255   0  43]
[  0 100   0   0   0   2   0 378   0  20]
[  0 265  15  37   0   4   0 179   0   0]
[  0 343   0  32   0   0   0  32   0  93]
[  8 287   0  75   6   1   4  67   3  49]
[  0 186   0  12   0   0   0 276   0  26]
[  0  67   0   0  45  66   0 266   0  56]
[  0  89   1  43   0   0   0 317   0  50]
[  0 164   0  14   8   9   0 234   0  71]
[  0 148   0  16   0   0   0 297   0  39]
[  0 170   0  15   0  12   0 277   0  26]
[  9   0 278   0   0   0 106   0 107   0]
[  0   0   0 180 134  52   0   0  38  96]
[  0  41 133 130   0   0   0  10  45 141]
[  0   0   0   0  76 218   0   6  66 134]
[  0  25   0   0 147 116   0  50  62 100]
[  0  29   0   0  15 163   0  56 152  85]
[  0   0   0   0  41  49  52  35 231  92]
[  0   0   0  25  62 260  37  22  69  25]
[  0   4  11 338   0 128   0   0  19   0]
[ 29  39   0   0   0 158   0  72  92 110]
[  0   0   5  50  18 144   0  73  

In [21]:
from sklearn.cluster import KMeans
data = n_mz

# 聚类为16类
kmeans = KMeans(n_clusters=16)
kmeans.fit(data)

# 输出每个样本所属的类别
labels = kmeans.labels_
for i in range(len(labels)):
    print(labels[i],end = ' ')
    if (i+1)%13 == 0:
        print('')

14 14 14 14 14 6 6 14 6 14 14 14 6 
1 7 7 11 0 7 12 11 6 15 7 10 10 
0 3 15 0 15 9 0 8 6 0 0 11 8 
3 3 0 15 14 8 0 15 15 15 10 14 0 
4 3 5 13 3 13 13 0 4 10 3 5 5 
12 1 1 1 1 12 11 9 1 13 12 12 12 
15 12 12 11 9 3 3 0 7 3 0 15 7 
0 0 3 3 10 1 0 3 11 13 10 3 5 
9 8 8 8 8 8 9 8 9 9 7 0 8 
11 8 1 4 11 3 7 7 13 14 1 5 9 
11 3 13 5 10 5 3 11 10 3 3 3 1 
6 4 15 13 11 4 15 13 4 13 9 3 4 
6 0 0 6 0 14 15 0 14 14 14 0 14 
1 11 13 9 3 15 3 11 1 5 11 12 3 
9 8 8 8 9 8 8 8 8 8 8 8 9 
2 2 2 2 2 2 2 2 2 12 12 12 12 


In [7]:
import os
import glob
import random
import jieba
from sklearn.cluster import KMeans
num_paragraphs = 200  # 抽取段落数
min_length = 500  # 每个段落最小长度
basepath = 'C:\\Users\\HUAWEI\\Desktop\\深度学习与自然语言处理\\第三次作业'  #文件夹位置
with open(basepath+'\\jyxstxtqj_downcc.com\\inf.txt','r',encoding='ANSI') as f:
    booklist = f.read().split(',') #读目录 
with open(basepath+"\\cn_stopwords.txt",'r',encoding = 'utf-8-sig') as f:
    stopwords = f.read().split() #读停用词
stopwords2 = [' ','wwwcrcom','www','w','c','r','cr173','com','□','说道','说','道','便','笑','=','txt','\n','本书来自www.cr173.com免费txt小说下载站','更多更新免费电子书请关注www.cr173.com','\u3000','目录']
stopwords.extend(stopwords2)
content = []
for book in booklist:
    path = basepath+'\\jyxstxtqj_downcc.com\\'+ book +'.txt'
    with open(path, 'r', encoding='ANSI') as f:
        data_txt = f.read()  # 读取文本文件
    words = list(data_txt)  #按照字取
    words = [i for i in words if i not in stopwords]
    pos = int(len(words)//13)
    for i in range(13):
        data_temp = []
        data_temp = data_temp + words[i*pos:i*pos+500]
        content.append(data_temp)

In [8]:
import numpy as np
import lda

# 208个段落，每个段落用一个list表示
paragraphs = content

# 构建词袋模型
vocab = set(word for paragraph in paragraphs for word in paragraph)
word2id = dict((word, id) for id, word in enumerate(vocab))
id2word = dict((id, word) for id, word in enumerate(vocab))
doc_word_matrix = np.zeros((len(paragraphs), len(vocab)), dtype=np.int32)
for i, paragraph in enumerate(paragraphs):
    for word in paragraph:
        doc_word_matrix[i, word2id[word]] += 1

# 训练LDA模型
model = lda.LDA(n_topics=10, n_iter=1000, random_state=1)
model.fit(doc_word_matrix)

# 输出每个主题的前10个关键词
topic_word = model.topic_word_
doc_topic = model.transform(doc_word_matrix)
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(list(vocab))[np.argsort(topic_dist)][:-(10+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

INFO:lda:n_documents: 208
INFO:lda:vocab_size: 3318
INFO:lda:n_words: 104000
INFO:lda:n_topics: 10
INFO:lda:n_iter: 1000
INFO:lda:<0> log likelihood: -1039783
INFO:lda:<10> log likelihood: -846716
INFO:lda:<20> log likelihood: -814996
INFO:lda:<30> log likelihood: -804420
INFO:lda:<40> log likelihood: -797704
INFO:lda:<50> log likelihood: -793247
INFO:lda:<60> log likelihood: -791286
INFO:lda:<70> log likelihood: -788452
INFO:lda:<80> log likelihood: -786100
INFO:lda:<90> log likelihood: -784662
INFO:lda:<100> log likelihood: -783358
INFO:lda:<110> log likelihood: -782162
INFO:lda:<120> log likelihood: -781297
INFO:lda:<130> log likelihood: -781063
INFO:lda:<140> log likelihood: -779191
INFO:lda:<150> log likelihood: -779499
INFO:lda:<160> log likelihood: -778123
INFO:lda:<170> log likelihood: -776691
INFO:lda:<180> log likelihood: -775707
INFO:lda:<190> log likelihood: -774777
INFO:lda:<200> log likelihood: -773986
INFO:lda:<210> log likelihood: -773999
INFO:lda:<220> log likelihood: 

Topic 0: 子 老 心 见 想 死 里 白 没 什
Topic 1: 手 中 出 身 时 声 两 心 然 三
Topic 2: 杨 女 龙 家 陈 玉 洛 鱼 英 花
Topic 3: 李 著 克 靖 苏 黄 郭 蓉 文 秀
Topic 4: 马 子 声 铁 两 长 见 身 少 里
Topic 5: 宝 韦 皇 公 兵 官 帝 中 宗 高
Topic 6: 剑 士 青 长 吴 范 名 蠡 招 国
Topic 7: 师 事 弟 山 兄 武 教 想 门 令
Topic 8: 十 中 王 国 书 三 回 写 年 生
Topic 9: 刀 中 著 子 萧 苗 袁 雄 夫 胡


In [10]:
data = doc_topic
# 聚类为16类
kmeans = KMeans(n_clusters=16)
kmeans.fit(data)
# 输出每个样本所属的类别
labels = kmeans.labels_
for i in range(len(labels)):
    print(labels[i],end = ' ')
    if (i+1)%13 == 0:
        print('')

9 12 15 15 15 4 4 15 15 15 15 15 15 
8 13 12 3 2 13 12 9 10 12 3 14 14 
3 5 2 10 10 3 1 9 3 10 5 12 11 
10 13 7 0 7 11 12 3 7 12 13 7 7 
12 0 0 0 0 7 0 0 0 0 7 9 0 
8 8 9 8 0 0 14 12 0 14 0 12 14 
12 9 9 4 3 4 4 4 10 10 15 4 9 
8 4 10 1 10 3 7 10 2 11 10 3 3 
11 11 11 11 11 11 11 2 3 9 13 3 3 
12 7 3 8 10 5 7 7 10 7 1 8 12 
9 13 10 7 1 7 10 10 2 1 10 10 2 
9 2 2 2 1 5 2 2 2 2 2 1 1 
9 5 5 5 2 10 5 5 2 5 5 10 5 
8 3 2 1 12 4 3 2 1 2 2 9 1 
5 5 5 3 5 5 5 5 5 5 5 5 5 
6 6 6 6 6 6 6 6 6 13 13 6 13 


In [11]:
import numpy as np
import random

doc_list=content
K=10
alpha=0.01
beta=0.1
num_iters=25
# 将列表中的元素转换为集合
word_dict = set(word for doc in doc_list for word in doc)
# 将列表中的元素转换为字典
word_dict = {word: i for i, word in enumerate(word_dict)}
# 将列表中的元素转换为numpy数组
doc_list = np.array([[word_dict[word] for word in doc] for doc in doc_list])
M = len(doc_list)  # 文档数
V = len(word_dict) # 词汇的大小
z = np.random.randint(0, K, size=(M, len(doc_list[0])))  # 主题分布
n_mz = np.zeros((M, K), dtype=int)  # 文档-主题分布
n_zv = np.zeros((K, V), dtype=int)  # 主题-词分布
n_z = np.zeros(K, dtype=int)  # 主题数量
for i in range(M):
    for j in range(len(doc_list[i])):
        k = z[i][j]
        n_mz[i][k] += 1
        n_zv[k][doc_list[i][j]] += 1
        n_z[k] += 1
# Gibbs采样
for iter in range(num_iters):
    for i in range(M):
        for j in range(len(doc_list[i])):
            k = z[i][j]
            n_mz[i][k] -= 1
            n_zv[k][doc_list[i][j]] -= 1
            n_z[k] -= 1
            p = (n_mz[i] + alpha) * (n_zv[:, doc_list[i][j]] + beta) / (n_z + V * beta)
            # 将代码向量化，以提高计算速度
            k = np.random.choice(K, p=p / p.sum())
            z[i][j] = k
            n_mz[i][k] += 1
            n_zv[k][doc_list[i][j]] += 1
            n_z[k] += 1

        
# 输出主题-词分布
for k in range(K):
    print('Topic {}:'.format(k+1))
    top5_positions = np.argsort(n_zv[k])[::-1][:5]
    for v in top5_positions:
        print('{}: {:.4f}'.format(list(word_dict.keys())[list(word_dict.values()).index(v)], n_zv[k][v] / n_z[k]))
print(n_mz)

Topic 1:
手: 0.0271
中: 0.0237
刀: 0.0193
身: 0.0141
见: 0.0127
Topic 2:
主: 0.0179
胡: 0.0166
家: 0.0146
子: 0.0142
声: 0.0138
Topic 3:
师: 0.0409
子: 0.0165
令: 0.0143
林: 0.0138
夫: 0.0135
Topic 4:
宝: 0.0161
子: 0.0144
韦: 0.0139
教: 0.0137
天: 0.0137
Topic 5:
马: 0.0270
女: 0.0208
回: 0.0176
十: 0.0173
中: 0.0142
Topic 6:
剑: 0.0800
士: 0.0254
国: 0.0176
青: 0.0173
手: 0.0158
Topic 7:
子: 0.0274
声: 0.0192
汉: 0.0134
姑: 0.0110
见: 0.0109
Topic 8:
时: 0.0185
天: 0.0149
手: 0.0145
石: 0.0144
两: 0.0141
Topic 9:
中: 0.0151
十: 0.0138
年: 0.0134
袁: 0.0115
三: 0.0108
Topic 10:
李: 0.0178
文: 0.0174
里: 0.0143
中: 0.0140
出: 0.0133
[[108   9   0 ...  26   0  37]
 [ 74  36   0 ...  53   0  95]
 [ 42  84   0 ...   7   0 305]
 ...
 [  0   0  34 ...  32   0  36]
 [ 15   0  35 ...  14   0   0]
 [  0  61   0 ...  18   3  55]]


In [12]:
from sklearn.cluster import KMeans
data = n_mz

# 聚类为16类
kmeans = KMeans(n_clusters=16)
kmeans.fit(data)

# 输出每个样本所属的类别
labels = kmeans.labels_
for i in range(len(labels)):
    print(labels[i],end = ' ')
    if (i+1)%13 == 0:
        print('')

12 4 0 0 0 0 0 0 4 0 0 0 0 
11 14 4 14 8 3 3 15 14 14 14 15 3 
4 13 8 1 1 1 1 1 8 1 4 6 9 
2 2 4 15 13 1 13 13 13 14 2 6 15 
12 7 7 7 7 7 7 7 7 15 7 15 15 
12 3 7 3 3 15 15 11 9 11 11 11 11 
12 15 15 4 13 14 14 14 13 13 4 4 15 
12 14 6 8 13 6 6 13 3 6 13 13 13 
4 9 9 1 3 9 9 9 12 15 2 13 9 
12 6 4 4 13 6 6 6 6 13 6 11 4 
12 7 6 9 6 6 6 6 6 6 6 6 3 
12 8 8 8 2 13 1 8 8 8 8 2 7 
12 3 1 1 3 13 9 1 3 14 14 1 1 
12 14 3 3 6 9 6 7 6 7 1 1 1 
5 5 5 5 8 13 5 5 13 14 5 5 5 
10 10 10 10 10 2 2 10 10 2 2 10 2 
